In [1]:
import pandas as pd
import talib as ta
import matplotlib.pyplot as plt
import os
from datetime import datetime, timedelta
import warnings
# warnings.warn('Error: A warning just appeared')
warnings.filterwarnings('ignore')
df = pd.DataFrame()

In [2]:
df = pd.read_csv("DailyData_18-22.csv")

In [3]:
df['datetime'] = pd.to_datetime(df['datetime'])
df.set_index('datetime', inplace=True)
df['EMA7'] = ta.EMA(df['close'], timeperiod=7)
df['EMA14'] = ta.EMA(df['close'], timeperiod=14)
df['EMA21'] = ta.EMA(df['close'],timeperiod=21)
df['ATR'] = ta.ATR(df['high'], df['low'], df['close'], timeperiod=14)
df['macd'], df['macdsignal'],df['histogram'] = ta.MACD(df['close'],12,26,9)
df.dropna(inplace = True)
df['signal'] = 0

In [4]:
def strategy(df):
    df['datetime'] = pd.to_datetime(df['datetime'])
    df.set_index('datetime', inplace=True)
    df['EMA7'] = ta.EMA(df['close'], timeperiod=7)
    df['EMA14'] = ta.EMA(df['close'], timeperiod=14)
    df['EMA21'] = ta.EMA(df['close'],timeperiod=21)
    df['ATR'] = ta.ATR(df['high'], df['low'], df['close'], timeperiod=14)
    df['macd'], df['macdsignal'],df['histogram'] = ta.MACD(df['close'],12,26,9)
    df.dropna(inplace = True)
    df['signal'] = 0
    inmarket = 0
    buy = 0
    open_price = -1
    # df['amount'] = 0
    df['sl'] = 0
    buy_sl = 0
    short_sl = 0
    c = 2
    for i in range(1,len(df)-1):
        if inmarket == 0:
            flag = 0
            # if(df['EMA7'][i-1]>df['EMA7'][i] and df['EMA7'][i+1] > df['EMA7'][i]+5 ) or (df['EMA7'][i-1]<df['EMA21'][i-1] and df['EMA7'][i+1]>df['EMA21'][i+1] ) :
            #     flag = 1
            if(df['EMA7'][i-1]>df['EMA7'][i] and df['EMA7'][i+1] > df['EMA7'][i]+5 and df['histogram'][i]>=0) or (df['EMA7'][i-1]<df['EMA21'][i-1] and df['EMA7'][i+1]>df['EMA21'][i+1] and df['histogram'][i]>=0) :
                flag = 1
            # if(df['EMA7'][i-1]<df['EMA21'][i-1] and df['EMA7'][i+1]>df['EMA21'][i+1]):
            #     flag = 1
            # if(df['EMA21'][i-1]<df['EMA21'][i] and df['EMA21'][i+1]+5 < df['EMA21'][i] ) or (df['EMA7'][i-1]>df['EMA21'][i-1] and df['EMA7'][i+1]<df['EMA21'][i+1] ):
            #     flag = 2
            elif(df['EMA21'][i-1]<df['EMA21'][i] and df['EMA21'][i+1]+5 < df['EMA21'][i] and df['histogram'][i]<0) or (df['EMA7'][i-1]>df['EMA21'][i-1] and df['EMA7'][i+1]<df['EMA21'][i+1] and df['histogram'][i]<0):
                flag = 2
            # if(df['EMA7'][i-1]>df['EMA21'][i-1] and df['EMA7'][i+1]<df['EMA21'][i+1]):
            #     flag = 2 
            if flag == 1:
                buy = 1 
                inmarket = 1 
                df['signal'][i] = 1
                buy_sl = df['close'][i] - c*df['ATR'][i]
                df['sl'][i] = buy_sl
                # open_price = df['open'][i]
            elif flag == 2:
                buy = 0
                inmarket = 1 
                df['signal'][i] = -1 
                short_sl = df['close'][i] + c*df['ATR'][i]
                df['sl'][i] = short_sl
                # open_price = df['open'][i]
        elif inmarket == 1:
            if buy == 1:
                # if df['close'][i]<= buy_sl:
                if (df['close'][i] <= df['sl'][i-1]) or (df['EMA14'][i-1]<df['EMA14'][i] and df['EMA14'][i+1] < df['EMA14'][i]) or (df['EMA7'][i-1]>df['EMA14'][i-1] and df['EMA7'][i+1]<df['EMA14'][i+1]):
                    inmarket = 0
                    df['signal'][i] = -1
                    df['sl'][i] = 0
                    buy_sl = 0
                else:
                    temp_sl_buy = df['close'][i]-c*df['ATR'][i]
                    buy_sl = max(df['sl'][i-1],temp_sl_buy)
                    # df['sl'][i] = 0
                    df['sl'][i] = buy_sl
                    # buy_sl = 0
            elif buy == 0:
                if (df['close'][i] >= df['sl'][i-1]) or (df['EMA7'][i-1]>df['EMA7'][i] and df['EMA7'][i+1] > df['EMA7'][i]) or (df['EMA7'][i-1]<df['EMA21'][i-1] and df['EMA7'][i+1]>df['EMA21'][i+1]):
                    inmarket = 0
                    df['signal'][i] = 1
                    df['sl'][i] = 0
                    short_sl = 0
                else:
                    temp_sl_short = df['close'][i]+c*df['ATR'][i]
                    short_sl = min(df['sl'][i-1], temp_sl_short)
                    # df['sl'][i] = 0
                    df['sl'][i] = short_sl

    df['sum'] = 0
    for i in range(1,len(df)):
        df['sum'][i] = df['sum'][i-1]+df['signal'][i-1]
    df.to_csv("Result_ATR_daily.csv")

In [5]:
# -*- coding: utf-8 -*-
"""
Created on Wed Dec 13 21:10:27 2023

@author: hp
"""

from backtesting import Backtest, Strategy
import pandas as pd

def count(df):
    # df = pd.DataFrame()
    # df = pd.read_csv("Result_ATR_daily.csv")
    df['datetime'] = pd.to_datetime(df['datetime'])
    df.set_index('datetime', inplace=True)
    cnt = 0
    for i in range(len(df)):
        if df['signal'][i]==1:
            cnt += 1
    print(cnt)
    return cnt
# df['pnl'] = 0
# for i in range(len(df)):
    
def modify_df(df):
    df['Close'] = df['close']
    df['Open'] = df['open']
    df['Low'] = df['low']
    df['High'] = df['high']
    df['Volume'] = df['volume']
    df['Open'] /= 1e6
    df['High'] /= 1e6
    df['Low'] /= 1e6
    df['Close'] /= 1e6
    return df

# df.set_index('datetime', inplace=True)
class MyStrat(Strategy):
    def init(self):
        pass
        # self.current_money = 500
        # self.lots = 0
        # self.TradeOpen = 'N'

    def next(self):
        if self.data['sum'] == 0:
            if self.data['signal'] == 1:
                self.buy()
            elif self.data['signal'] == -1:
                self.sell()
        else:
            if self.data['signal'] == -1 or self.data['signal']== 1:
                self.position.close()
        # if self.data['sum'] == 0:
        #     if self.data['signal'] == 1:
        #         self.buy()
        # else:
        #     if self.data['signal'] == -1:
        #         self.position.close()
        
        
        
        # if self.data['signal'] == 1:
        #     if self.TradeOpen == 'N':
        #         self.buy()
        #         self.TradeOpen = 'Y'
        #     elif self.TradeOpen == 'Y':
        #         self.position.close()
        #         self.TradeOpen = 'N'
        # elif self.data['signal'] == -1:
        #     if self.TradeOpen == 'N':
        #         self.sell()
        #         self.TradeOpen = 'Y'
        #     elif self.TradeOpen == 'Y':
        #         self.position.close()
        #         self.TradeOpen = 'N'
        # if self.data['sum'] == 0:
        #     self.position.close()
        #     if self.data['signal'] == 1:
        #         self.buy()
        #     elif self.data['signal'] == -1:
        #         self.sell()

def backtest(df):
    bt = Backtest(df, MyStrat, cash=1000, commission=0.001,trade_on_close=True)
    result = bt.run()
    bt.plot(superimpose=False)
    return bt
    # bt.plot()

Loading BokehJS ...

83


GridPlot(id='p1283', ...)

In [6]:
result

Start                     2018-02-03 05:30:00
End                       2022-01-12 05:30:00
Duration                   1439 days 00:00:00
Exposure Time [%]                   49.027778
Equity Final [$]               1164776.837475
Equity Peak [$]                 1167932.67269
Return [%]                      116377.683747
Buy & Hold Return [%]              377.204901
Return (Ann.) [%]                   498.69738
Volatility (Ann.) [%]              258.322631
Sharpe Ratio                         1.930521
Sortino Ratio                       30.156478
Calmar Ratio                        52.629464
Max. Drawdown [%]                   -9.475631
Avg. Drawdown [%]                   -2.651792
Max. Drawdown Duration      100 days 00:00:00
Avg. Drawdown Duration       10 days 00:00:00
# Trades                                   83
Win Rate [%]                        89.156627
Best Trade [%]                      75.320887
Worst Trade [%]                     -2.725318
Avg. Trade [%]                    

In [7]:
def handwritten_backtest(df):
    df['pnl'] = 0
    df['days'] = 0
    d = 0
    entry_price = 0
    pos_pnl = 0
    neg_pnl = 0
    total_pnl = 0
    NoWinningTrade = 0
    NoLossingTrade = 0
    for i in range(len(df)-1):
        if df['sum'][i] == 0:
            d = 0
            if df['signal'][i] == 1:
                entry_price = df['open'][i+1]
            elif df['signal'][i] == -1:
                entry_price = df['open'][i+1]
        else:
            if df['signal'][i] == 1:
                df['pnl'][i] = (entry_price - df['close'][i])*(1000/entry_price)
                df['days'][i] = d+2
            elif df['signal'][i] == -1:
                df['pnl'][i] = (df['close'][i]-entry_price)*(1000/entry_price)
                df['days'][i] = d+2
            else:
                d += 1
    import sys
    max_int = sys.maxsize
    MaxLossingTrade = max_int
    min_int = -sys.maxsize - 1
    MaxWinningTrade = min_int
    for i in range(len(df)):
        if df['pnl'][i] >0:
            pos_pnl += df['pnl'][i]
            NoWinningTrade += 1
            MaxWinningTrade = max(MaxWinningTrade,df['pnl'][i])
        elif df['pnl'][i]<0:
            neg_pnl += df['pnl'][i]
            NoLossingTrade += 1
            MaxLossingTrade = min(MaxLossingTrade,df['pnl'][i])

    print("Positive PnL is",pos_pnl)
    print("negative PnL is",neg_pnl)
    print("total PnL(Gross Profit) is", pos_pnl+neg_pnl)
    print("Win[%] is", NoWinningTrade/(NoWinningTrade+NoLossingTrade)*100)
    print("Largest Winning Trade is",MaxWinningTrade)
    print("Largest Lossing Trade is",MaxLossingTrade)
    print("Avg. winning trade is",pos_pnl/NoWinningTrade)
    print("Avg. Lossing trade is",neg_pnl/NoLossingTrade)
    df.to_csv("ResultWithPnLUpdated.csv")

Positive PnL is 8069.318150341412
negative PnL is -134.99430719962257
total PnL(Gross Profit) is 7934.323843141789
Win[%] is 89.1566265060241
Largest Winning Trade is 754.9604365109129
Largest Lossing Trade is -26.27899648025375
Avg. winning trade is 109.04483986947854
Avg. Lossing trade is -14.99936746662473


PermissionError: [Errno 13] Permission denied: 'ResultWithPnLUpdated.csv'

In [96]:
df['days'].max()

50

In [97]:
df['pnl'].min()

-26.27899648025375

In [98]:
pnls = []
for i in range(len(df)):
    if df['pnl'][i] != 0:
        pnls.append(df['pnl'][i])
pnls.sort()

In [99]:
pnls

[-26.27899648025375,
 -25.023967793650304,
 -24.872420006965726,
 -21.26350779859113,
 -16.35773864689527,
 -13.854792561945157,
 -9.099932362145696,
 -8.982990976332951,
 -8.064121517511204,
 -4.095555672323087,
 -1.5138956439535918,
 -1.199992682971352,
 0.3544039907676434,
 1.7882381688885587,
 3.8519907413134327,
 5.297174435450945,
 5.963569281929278,
 7.91704426313089,
 9.962599243236166,
 10.448898966580956,
 14.10212870677989,
 14.92076694751299,
 17.183730841989977,
 18.007148343445753,
 18.095534249471154,
 19.217816804768614,
 19.27055539591971,
 19.30801783278655,
 20.515254688946236,
 20.789409812633863,
 21.699774272424424,
 22.22844102513351,
 22.568812761414033,
 27.347610191442516,
 27.720368213295167,
 28.304317114721407,
 30.86720390766808,
 31.12000573594292,
 31.34832657041225,
 31.40641522735285,
 34.14510568139454,
 34.578590035390356,
 35.11948519426532,
 35.56377620722863,
 36.80462272075092,
 38.55633839811217,
 40.31290943608121,
 41.20732117640138,
 44.39549